# Data Analysis Extra - Playlists

Como análisis extra pensé que podría analizar las plasyslist que crearon lso usuarios durante la pandemia, podría dar indicadores de qué tipo de música prefiere escuchar durante una pandemia.

El proceso a realizar es buscar las playlist que tengan en el nombre alguna palabra relacionada con la pandemia, me he limitado al español

In [95]:
# Ver : https://towardsdatascience.com/what-music-do-people-listen-to-during-the-coronavirus-a-report-using-data-science-1a2035d12430
# Leer: https://blog.chartmetric.com/covid-19-effect-on-the-global-music-business-part-1-genre/
# https://gist.github.com/ilias1111/e503bbab0a98c20377686cc75ffad451

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import seaborn as sns
from datetime import datetime

# Meter credenciales de Spotify
passw = pd.read_csv("pass_spotify.txt", sep = ',', encoding="utf-8")
client_id = passw.columns[0]
client_secret = passw.columns[1]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Vemos como está montada la estructura de información en la API de Spotify, para buscar una palabra en los nombres de las playlists
print(sp.search('q=fernando simón', type='playlist'))
# ["covid","cuarentena", 'pandemia','covid19', "coronavirus", 'fernando simón']

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=q%3Dfernando+sim%C3%B3n&type=playlist&offset=0&limit=10', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.co

In [7]:
# Creamos un script ya completo para buscar las palabras y sacar solo el total de listas que contiene esta palabra en alguna parte de su información
for word in ["fernando simón"]:
    print(sp.search('q="{}"'.format(word), type='playlist')['playlists']['items'])


[{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/01bawF2DM0YzefGbC54Ua1'}, 'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1', 'id': '01bawF2DM0YzefGbC54Ua1', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb80c823de1e04b1fe7f2073ce', 'width': None}], 'name': 'la playlist que escucha fernando simón', 'owner': {'display_name': 'mikenuggets', 'external_urls': {'spotify': 'https://open.spotify.com/user/079077tae8iz2j9qk6431p9s2'}, 'href': 'https://api.spotify.com/v1/users/079077tae8iz2j9qk6431p9s2', 'id': '079077tae8iz2j9qk6431p9s2', 'type': 'user', 'uri': 'spotify:user:079077tae8iz2j9qk6431p9s2'}, 'primary_color': None, 'public': None, 'snapshot_id': 'Mjc0LGVlNGQ1YTY3YWIzNGZmMjhiYzBiYzcxM2MxZDBkZWY4YzBjNDhlNmU=', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/01bawF2DM0YzefGbC54Ua1/tracks', 'total': 271}, 'type': 'playlist', 'uri': 'spotify:playlist:01bawF2DM0YzefGbC54Ua1'}

In [12]:
# Extracción de las playlist con dichos nombres
# Documentación: https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlist/
# Función para extrar determinados features de la playlist
# ['items']["tracks"]['items'][0]["added_at"]
def batch_proccess(x,lista):
    for i in x['playlists']['items']:
        lista.append({"name" : i['name'], "total":i["tracks"]['total'], 'id':i['id'], "uri":i["uri"], "URL":i["tracks"]['href']})

# Sacar un listado de las playlist, en bloques de 50, que es el máximo que te permite la API

Time1 = datetime.now()

list_of_playlists = []
for term in ["covid","pandemia","cuarentena",'covid19', "coronavirus", 'fernando simón']:
    for i in range(0,2000, 50):
        try:
            init_data = sp.search('q="{}"'.format(term), type='playlist', limit=50, offset=i)
            batch_proccess(init_data,list_of_playlists)
        except:
            print("Error")
Time2 = datetime.now()

print("Tiempo ejecución:", Time2 -Time1)

Tiempo ejecución: 0:00:21.936139


In [39]:
# Meto las playlista en dataframe y echo un ojo
playlists = pd.DataFrame(list_of_playlists).drop_duplicates().reset_index(drop=True)
playlists.head(20)
playlists= playlists[0:10] #PROBANDO!!!!!
playlists

,name,total,id,uri,URL
0,quarantine Vibes | Covid 19 |,42,6zjC569rXoYGOLwPUt37WW,spotify:playlist:6zjC569rXoYGOLwPUt37WW,https://api.spotify.com/v1/playlists/6zjC569rX...
1,Calma,151,37i9dQZF1DWY5LGZYBBHHz,spotify:playlist:37i9dQZF1DWY5LGZYBBHHz,https://api.spotify.com/v1/playlists/37i9dQZF1...
2,COVID-19 Quarantine Party,29,1Tn6OrkJhOYO5u6y16erqB,spotify:playlist:1Tn6OrkJhOYO5u6y16erqB,https://api.spotify.com/v1/playlists/1Tn6OrkJh...
3,Canciones Mas Virales que el Coronavirus 🦠🎶,104,4Y2h9TZNZersxSYeXujCNu,spotify:playlist:4Y2h9TZNZersxSYeXujCNu,https://api.spotify.com/v1/playlists/4Y2h9TZNZ...
4,Drunksouls COVID19 quarantine discovery: music...,49,64KUBwKGkUEqIU08HDhJ94,spotify:playlist:64KUBwKGkUEqIU08HDhJ94,https://api.spotify.com/v1/playlists/64KUBwKGk...
5,chill beats to quarantine to,124,5bPe4ZISvKWU5yTzsSO9lX,spotify:playlist:5bPe4ZISvKWU5yTzsSO9lX,https://api.spotify.com/v1/playlists/5bPe4ZISv...
6,Quarantine covid - 19,100,7qJzyN6atsM49NOWnqGbEK,spotify:playlist:7qJzyN6atsM49NOWnqGbEK,https://api.spotify.com/v1/playlists/7qJzyN6at...
7,MÁS PERREO MENOS COVID,99,1RYr3ngXNoPVtl7blr9Kaf,spotify:playlist:1RYr3ngXNoPVtl7blr9Kaf,https://api.spotify.com/v1/playlists/1RYr3ngXN...
8,CORONAVIRUS QUARANTINE PARTY COVID-19 2020,55,3r9u5bXzAYEvc0xBKkSnAx,spotify:playlist:3r9u5bXzAYEvc0xBKkSnAx,https://api.spotify.com/v1/playlists/3r9u5bXzA...
9,Plenas que no pudimos bailar por COVID,78,2U4J1vAVC8DvV3fPfrgXVy,spotify:playlist:2U4J1vAVC8DvV3fPfrgXVy,https://api.spotify.com/v1/playlists/2U4J1vAVC...


In [105]:
'''for word in ["corona"]:
    print(sp.search('q="{}"'.format(word), type='playlist')['playlists']['items'][4])
'''
init_data = sp.playlist_tracks('spotify:playlist:4Y2h9TZNZersxSYeXujCNu')

init_data['items'][0]['track']


'JVKE'

In [109]:
#FALTA JUNTAR ARTISTAS CON NOMBRES Y VER CÓMO LO JUNTAMOS EN AL DTAFRAME FINAL
# Sacar las canciones de las listas para tenerlas todas juntas
# Tarda bastante tiempo, alrededor de XX

# Para sacar canciones
Time1 = datetime.now()
songs = []
for uri in playlists['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    songs.append(k['track']["id"])
            except:
                pass

Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)


# Para sacar artistas
Time1 = datetime.now()
artists = []
for uri in playlists['uri']:

        lenght = sp.playlist_tracks(uri)['total']
        for i in range(0,lenght, 50):
            init_data = sp.playlist_tracks(uri, limit=50, offset=i)
            try:
                for k in init_data['items']:
                    artists.append(k['track']['artists'][0]['name'])
            except:
                pass

Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)

print(len(songs))

Tiempo ejecución: 0:00:02.686349
Tiempo ejecución: 0:00:02.695148
832


In [20]:
#Vamos a hacer un de las canciones más repetidas
from collections import Counter
data_playlist_songs = pd.DataFrame.from_dict(Counter(songs), orient='index').reset_index().rename(columns={"index":"uri", 0:"count"})
data_playlist_songs.sort_values(by=['count'], ascending=False)

,id,count
6747,None,603
243,2DEZmgHKAvm41k4J3R2E9Y,563
320,0SqqAgdovOE24BzxIClpjw,522
760,7k4t7uLgtOxPwTpFmtJNTY,427
2240,0VjIjW4GlUZAMYd2vXMi3b,379
...,...,...
118089,22v9ZVkWWAYjderL7ZWuuH,1
118090,3deUFJzGwnWTsTel4EeEeV,1
118091,4IRpbTrTCHkP3aeRO48tZs,1
118093,0PZmgHEwlQMfEEg80MYHEq,1


In [23]:
# Hay un test previo de extracción
test_list = songs[0:200]

In [25]:
# Función para sacar las features de canciones que me interesan desde la id
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  valence = features[0]['valence']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  id = features[0]['id']

  track = [name, album, artist, release_date, length, popularity,
           acousticness, danceability, energy, instrumentalness,
           liveness, loudness, speechiness, valence, tempo, time_signature, id]
  return track


# Loop para sacar features de todas las canciones de una lista
# OJO, tarda bastante dependendo del número de canciones, por lo que antes hago un testeo
import time
list_toextract = test_list

tracks = []

Time1 = datetime.now()


for i in range(len(list_toextract)):
    time.sleep(.5)
    track = getTrackFeatures(list_toextract[i])
    tracks.append(track)
    
Time2 = datetime.now()

print("Tiempo 1:", Time2 -Time1)
print("Tiempo 2:", Time2_v -Time1_v)


Tiempo 1: 0:01:58.621202
Tiempo 2: 0:00:38.930047


In [29]:
# Metemos la información en Dataframes
data_features = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date',
                                             'length', 'popularity','acousticness', 'danceability', 'energy',
                                             'instrumentalness', 'liveness', 'loudness',
                                             'speechiness', 'valence','tempo', 'time_signature', 'id'])

data_features_final = data_features [['name','artist','album','release_date','length', 'popularity',
                                             'acousticness', 'danceability', 'energy',
                                             'instrumentalness', 'liveness', 'loudness',
                                             'speechiness', 'valence','tempo', 'time_signature', 'id']]

data_features_final = data_features_final.rename(columns = {'id':'spotify_id'})


# Finalmente meto el año de Release date (lanzamiento de la canción) en un nueva columna que me va a vernir en uno de los análisis
data_features_final['release_date_year'] = pd.to_datetime(data_features_final['release_date'])
data_features_final['release_date_year'] = pd.DatetimeIndex(data_features_final['release_date']).year
print(data_features_final.shape)
data_features_final.head(4)

#Normalizamos features
features_to_normalize = ['length', 'popularity', 'loudness', 'tempo', 'speechiness']

data_features_final[features_to_normalize] = data_features_final[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))
data_features_final.head(2)

(400, 18)


,name,artist,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,spotify_id,release_date_year
0,Mess Around (feat. B00sted),Enrey,Mess Around (feat. B00sted),2018-11-05,0.618749,0.000000,0.0969,0.643,0.842,0.0,0.1030,0.939463,0.325221,0.131,0.407541,4,2uNXNNIs1gpxVac4s9oW2s,2018
1,Show You,JJ Young,Show You,2019-09-02,0.351681,0.202247,0.0651,0.584,0.779,0.0,0.0584,0.922450,0.469027,0.216,0.869992,4,3NodDyiiU5ncq4EHsdE6Tf,2019


## Saco el genre también

In [31]:
# Extracción de genres
# Esta celda tarda en ejecutarse alrededor de 1 minuto
#Seleccionar cuantos datos se van a sacar
list_toextract_artist = test_list_artist

# Función para extraerlos, en lotes de 50 porque es el máximo que te deja la API de Spotify
def extract_artist(artist_list):
    count_artist= 0
    data = ['test11111111', 'tes111111', 'test111111', 0, 0]
    df_final = pd.DataFrame([data], columns=["id", "name", "genres", 'followers', 'popularity'])
    
    API_LIMIT = 50
    search_list = []

    for artist in artist_list:
        count_artist += 1
        search_results = sp.search(q = 'artist:"{}"'.format(artist), type="artist",  limit=API_LIMIT)
        
        count_oneartist= 0 # Se añade un contador par quedarnos sólo con la primera búsqueda que he comprobado que siempre es la buena.
        for t in search_results['artists']['items']:
            count_oneartist += 1
            if count_oneartist ==2:
                break
            else:
                search_list += [[t["id"], t["name"], t["genres"], t['followers']['total'], t['popularity']]]
        
        df_search = pd.DataFrame(search_list, columns=["id", "name", "genres", 'followers', 'popularity'])
        
        df_final = df_final.append(df_search) # El problema de esto es que crea duplicados pero no nos afecta en lo que necesitamos
    print('Artist count:', count_artist)  
    return df_final

# Extracción de los datos
Time1 = datetime.now()
df_genres = extract_artist(list_toextract_artist)
Time2 = datetime.now()
print("Tiempo ejecución:", Time2 -Time1)

print(df_genres.shape)
df_genres.head(6)

########



# Tarda en ejecutarse dependiendo del número de artistas, menos de 1 minuto
# Creamos una copia de la columna genres para mantenerla y la trasnformo a strings
df_genres['genres_together'] = df_genres['genres'].copy()
df_genres['genres_together'] = [','.join(i) if isinstance(i, list) else i for i in df_genres['genres_together']]

# Como la columna genres es una lista con los diferentes géneros, he preferido separarlo en columnas y mantener ciertas columnas

df_genres2 = pd.concat([df_genres['genres'].apply(pd.Series), df_genres[['genres_together',
                                                                         'id',
                                                                         "name",
                                                                         'followers',
                                                                         'popularity']]], axis = 1)
# Quitamos los artistas duplicados
df_genres3 = df_genres2.drop_duplicates(df_genres2.columns[~df_genres2.columns.isin(['id'])])

#Renombramos columnas
df_genres4 = df_genres3.rename(columns={0:'genre1',
                   1:'genre2',
                   2:'genre3',
                   3:'genre4',
                   4:'genre5',
                   5:'genre6',
                   6:'genre7',
                   7:'genre8',
                   8:'genre9',
                   9:'genre10',
                   'genres_together':'genres_together',
                   'id':'artist_id',
                   'name':'Artist',
                   'followers':'artist_followers',
                   'popularity':'artist_popularity'}).copy()
print(df_genres4.shape)
print(len(df_genres4['Artist'].unique()))
df_genres4.head(10)


NameError: name 'test_list_artist' is not defined

In [30]:
# Export songs list
data_glo = pd.merge(data_features, df_genres7, on=['Artist'], how='inner')

data_features_final.to_csv("data_ana_playlist_songs.csv", sep = ',')